In [3]:
!pip install --upgrade paddlehub -i https://pypi.tuna.tsinghua.edu.cn/simple

In [4]:
import paddlehub as hub
import paddle

In [ ]:
#查看Paddle、PaddleHub版本以及模型信息
print(paddle.__version__)
!pip show paddlehub

model = hub.Module(name="ernie", task='seq-cls', num_classes=14) # 在多分类任务中，num_classes需要显式地指定类别数，此处根据数据集设置为14
print(model)

In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment.
%cd ./data/data16287/
!tar -zxvf thu_news.tar.gz
!ls -hl thu_news

print("题目二：打印验证集前3条数据")
!head -n 4 thu_news/valid.txt
print("题目二：打印验证集后3条数据")
!tail -n 3 thu_news/valid.txt

In [ ]:
import os, io, csv
from paddlehub.datasets.base_nlp_dataset import InputExample, TextClassificationDataset

# 数据集存放位置
DATA_DIR="./data/data16287/thu_news"

In [ ]:
class ThuNews(TextClassificationDataset):
    def __init__(self, tokenizer, mode='train', max_seq_len=128):
        if mode == 'train':
            data_file = 'train.txt'
        elif mode == 'test':
            data_file = 'test.txt'
        else:
            data_file = 'valid.txt'
        super(ThuNews, self).__init__(
            base_path=DATA_DIR,
            data_file=data_file,
            tokenizer=tokenizer,
            max_seq_len=max_seq_len,
            mode=mode,
            is_file_with_header=True,
            label_list=['体育', '科技', '社会', '娱乐', '股票', '房产', '教育', '时政', '财经', '星座', '游戏', '家居', '彩票', '时尚'])

    # 解析文本文件里的样本
    def _read_file(self, input_file, is_file_with_header: bool = False):
        if not os.path.exists(input_file):
            raise RuntimeError("The file {} is not found.".format(input_file))
        else:
            with io.open(input_file, "r", encoding="UTF-8") as f:
                reader = csv.reader(f, delimiter="\t", quotechar=None)
                examples = []
                seq_id = 0
                header = next(reader) if is_file_with_header else None
                for line in reader:
                    example = InputExample(guid=seq_id, text_a=line[0], label=line[1])
                    seq_id += 1
                    examples.append(example)
                return examples

train_dataset = ThuNews(model.get_tokenizer(), mode='train', max_seq_len=128)
dev_dataset = ThuNews(model.get_tokenizer(), mode='dev', max_seq_len=128)
test_dataset = ThuNews(model.get_tokenizer(), mode='test', max_seq_len=128)
for e in train_dataset.examples[:3]:
    print(e)

In [ ]:
optimizer = paddle.optimizer.Adam(learning_rate=5e-5, parameters=model.parameters())  # 优化器的选择和参数配置
trainer = hub.Trainer(model, optimizer, checkpoint_dir='./ckpt', use_gpu=True)        # fine-tune任务的执行者

In [ ]:
trainer.train(train_dataset, epochs=3, batch_size=32, eval_dataset=dev_dataset, save_interval=1)   # 配置训练参数，启动训练，并指定验证集

In [ ]:
result = trainer.evaluate(test_dataset, batch_size=32)    # 在测试集上评估当前训练模型